Imports et chargement des données

In [2]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap
sys.path.append("..")
from src.data_prep import build_datasets
from src.model_utils import split_train_valid

In [3]:
import mlflow.sklearn

MODEL_DIR = Path("../model")
final_pipeline = mlflow.sklearn.load_model(MODEL_DIR)
final_pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['SK_ID_CURR', 'CNT_CHILDREN',
                                                   'AMT_INCOME_TOTAL',
                                                   'AMT_CREDIT', 'AMT_ANNUITY',
                                                   'AMT_GOODS_PRICE',
                                                   'REGION_POPULATION_RELATIVE',
                                                   'DAYS_BIRTH',
                                                   'DAYS_EMPLOYED',
                                                   'DAYS_REGISTRATION',
                                                   'DAYS_ID_PUBLISH',
                                                   'OWN_CAR_AGE', 'FLAG_MOBIL',
                                                   'FLAG_EMP_PHONE',
                                                   'FLAG_WORK_PHON...
                                                   'NAME_TYPE_SUITE',
                                                   'NAME_INCOME_TYPE',
                                                   'NAME_EDUCATION_TYPE',
                                                   'NAME_FAMILY_STATUS',
                                                   'NAME_HOUSING_TYPE',
                                                   'OCCUPATION_TYPE',
                                                   'WEEKDAY_APPR_PROCESS_START',
                                                   'ORGANIZATION_TYPE',
                                                   'FONDKAPREMONT_MODE',
                                                   'HOUSETYPE_MODE',
                                                   'WALLSMATERIAL_MODE',
                                                   'EMERGENCYSTATE_MODE'])])),
                ('model',
                 RandomForestClassifier(class_weight='balanced_subsample',
                                        n_estimators=200, n_jobs=-1,
                                        random_state=42))])

In [4]:
import joblib
MODEL_PKL = Path("../model/model.pkl")
final_pipeline = joblib.load(MODEL_PKL)
final_pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['SK_ID_CURR', 'CNT_CHILDREN',
                                                   'AMT_INCOME_TOTAL',
                                                   'AMT_CREDIT', 'AMT_ANNUITY',
                                                   'AMT_GOODS_PRICE',
                                                   'REGION_POPULATION_RELATIVE',
                                                   'DAYS_BIRTH',
                                                   'DAYS_EMPLOYED',
                                                   'DAYS_REGISTRATION',
                                                   'DAYS_ID_PUBLISH',
                                                   'OWN_CAR_AGE', 'FLAG_MOBIL',
                                                   'FLAG_EMP_PHONE',
                                                   'FLAG_WORK_PHON...
                                                   'NAME_TYPE_SUITE',
                                                   'NAME_INCOME_TYPE',
                                                   'NAME_EDUCATION_TYPE',
                                                   'NAME_FAMILY_STATUS',
                                                   'NAME_HOUSING_TYPE',
                                                   'OCCUPATION_TYPE',
                                                   'WEEKDAY_APPR_PROCESS_START',
                                                   'ORGANIZATION_TYPE',
                                                   'FONDKAPREMONT_MODE',
                                                   'HOUSETYPE_MODE',
                                                   'WALLSMATERIAL_MODE',
                                                   'EMERGENCYSTATE_MODE'])])),
                ('model',
                 RandomForestClassifier(class_weight='balanced_subsample',
                                        n_estimators=200, n_jobs=-1,
                                        random_state=42))])

Construction d’un jeu de validation pour SHAP

In [5]:
# On recharge les données préparées
train_df, test_df = build_datasets()

# Split train / valid pour avoir des exemples récents en mémoire
X_train, X_valid, y_train, y_valid = split_train_valid(train_df, test_size=0.2, random_state=42)

X_valid.shape, y_valid.shape

((61503, 192), (61503,))

Préparation des données pour SHAP

On récupère les étapes internes du pipeline : préprocesseur + modèle arbre.

In [6]:
# On suppose que final_pipeline est un Pipeline(preprocess, model)
preprocess = final_pipeline.named_steps["preprocess"]
model = final_pipeline.named_steps["model"]
# Pour SHAP, on prend un sous-échantillon de X_valid (par ex. 2000 lignes max)
X_sample = X_valid.sample(n=min(2000, len(X_valid)), random_state=42)
# Transformation (imputation + OneHotEncoder)
X_trans = preprocess.transform(X_sample)
# Noms de features après OneHot
feature_names = preprocess.get_feature_names_out()
X_trans.shape, len(feature_names)

((2000, 316), 316)

SHAP global (summary plot)

In [ ]:
# Explainer SHAP pour modèle de type arbre (RandomForest)
explainer = shap.TreeExplainer(model)
# Calcul des valeurs SHAP
shap_values = explainer(X_trans)
# Summary plot global : importance moyenne des variables
plt.figure(figsize=(10, 6))
shap.summary_plot(
    shap_values.values,
    X_trans,
    feature_names=feature_names,
    show=False
)
plt.title("Importance globale des variables (SHAP)")
plt.tight_layout()

# Sauvegarde dans reports/figures/
Path("../reports/figures").mkdir(parents=True, exist_ok=True)
plt.savefig("../reports/figures/shap_global.png", bbox_inches="tight")
plt.show()

SHAP local pour un client

In [ ]:
# On prend un indice arbitraire (par ex. le premier de X_trans)
idx = 0
# Valeurs SHAP pour ce client
shap_values_one = shap_values[idx]
# On peut aussi garder les features brutes pour interpréter
client_raw = X_sample.iloc[idx]
client_raw.head()

In [ ]:
# Waterfall plot local (impact des features sur la prédiction de ce client)
plt.figure(figsize=(8, 6))
shap.plots.waterfall(shap_values_one, max_display=15, show=False)
plt.title("Explication locale SHAP pour un client")
plt.tight_layout()
plt.savefig("../reports/figures/shap_local.png", bbox_inches="tight")
plt.show()